In [33]:
import requests
import pandas as pd

In [34]:
# this function will accept a FIADB-API fullreport URL and return dataframes for the estimates as well as subtotals, and totals where available.
def fiadb_api_GET(url):
    # make request
    resp = requests.get(url)
    # parse response to json
    data = resp.json()

    # create output dictionary and populate it with estimate data frames
    outDict = {}
    # append estimates
    outDict['estimates'] = pd.DataFrame(data['estimates'])
    """

    # append subtotals and totals if present
    if 'subtotals' in data.keys():
        subT = {}
        for i in data['subtotals'].keys():
            subT[i] = pd.DataFrame(data['subtotals'][i])
        outDict['subtotals'] = subT
        outDict['totals'] = pd.DataFrame(data['totals'])

    # append metadata
    outDict['metadata'] = data['metadata']
    """
    return outDict

In [35]:
# example of usage
url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=County%20code%20and%20name&cselected=Ownership%20class&schema=FS_FIADB.&snum=2&wc=342021&outputFormat=NJSON"
getData = fiadb_api_GET(url=url)

In [36]:
# estimate data frame
getData['estimates']
df = getData['estimates']




In [37]:
df = df.drop(["PLOT_COUNT", "SE", "SE_PERCENT", "VARIANCE"], axis = 1)
df["GRP2"] = df["GRP2"].str.split(" ", n = 1).str[1]
df["GRP1"] = df["GRP1"].str.split(" ", n = 1).str[1]


In [38]:
df = df.pivot(index='GRP1', columns='GRP2', values='ESTIMATE')


In [39]:
df['Total'] = df.sum(axis=1)

In [40]:
df.loc['Total']= df.sum()

In [41]:
df = df.round(0)

In [42]:
df.head()

GRP2,County and Municipal,Dept of Defense,Fish and Wildlife Service,National Park Service,Other federal,Other local govt,Private,State,Total
GRP1,,,,,,,,,
34001 NJ Atlantic,46433.0,NaN,507.0,NaN,3571.0,NaN,73626.0,59466.0,183602.0
34003 NJ Bergen,3234.0,NaN,NaN,NaN,NaN,NaN,4679.0,9932.0,17845.0
34005 NJ Burlington,10965.0,19626.0,NaN,NaN,NaN,NaN,105640.0,143447.0,279678.0
34007 NJ Camden,20728.0,NaN,NaN,NaN,NaN,NaN,10106.0,10918.0,41752.0
34009 NJ Cape May,300.0,NaN,13284.0,NaN,NaN,NaN,16647.0,30521.0,60752.0


In [45]:
df = df.fillna(0)
df = df.round(0)
df = df.astype(int)

df.head()

GRP2,County and Municipal,Dept of Defense,Fish and Wildlife Service,National Park Service,Other federal,Other local govt,Private,State,Total
GRP1,,,,,,,,,
34001 NJ Atlantic,46433,0,507,0,3571,0,73626,59466,183602
34003 NJ Bergen,3234,0,0,0,0,0,4679,9932,17845
34005 NJ Burlington,10965,19626,0,0,0,0,105640,143447,279678
34007 NJ Camden,20728,0,0,0,0,0,10106,10918,41752
34009 NJ Cape May,300,0,13284,0,0,0,16647,30521,60752


In [46]:
df = df.reset_index()

In [47]:
df = df.rename(columns={'GRP1':'County Code and Name'})

In [48]:
df=df[['County Code and Name', 'Total', 'National Park Service', 'Fish and Wildlife Service', 'Dept of Defense', 'Other federal', 'State', 'County and Municipal', 'Other local govt', 'Private']]

In [49]:
df = df.reindex(index=[21, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [50]:
df.columns = [''.join(str(s) for s in col if s) for col in df.columns]

In [51]:
df = df.replace(0,"-")

In [52]:
df = df.style.hide(axis='index')

In [56]:
df


County Code and Name,Total,National Park Service,Fish and Wildlife Service,Dept of Defense,Other federal,State,County and Municipal,Other local govt,Private
Total,1967004,29469,23769,54126,8534,634517,326434,3333,886823
34001 NJ Atlantic,183602,-,507,-,3571,59466,46433,-,73626
34003 NJ Bergen,17845,-,-,-,-,9932,3234,-,4679
34005 NJ Burlington,279678,-,-,19626,-,143447,10965,-,105640
34007 NJ Camden,41752,-,-,-,-,10918,20728,-,10106
34009 NJ Cape May,60752,-,13284,-,-,30521,300,-,16647
34011 NJ Cumberland,148801,-,-,-,-,66973,8516,-,73312
34013 NJ Essex,7971,-,-,-,-,4349,3623,-,-
34015 NJ Gloucester,61573,-,-,-,3178,2856,5545,-,49994
34017 NJ Hudson,1517,-,-,-,-,1517,-,-,-
